In [1]:
import numpy as np
from classy import Class

### Evaluate derived parameters $h$, $\sigma_8$ using _Plank18_ value for $\theta_*$

In [3]:
def derived_parameters(p):
    #set parameters to configure CLASS computations
    params = {
        'omega_b': p[0],
        'omega_cdm': p[1],
        'A_s': p[2], 
        'n_s': p[3], 
        'N_ur': 2.0328,
        'N_ncdm' : 1.0,
        'omega_ncdm' : 0.0006442,
        '100*theta_s' : 1.041533, #from Summit paper
        'recombination' : 'HyRec',
        'tau_reio' : 0.0544,
        'non linear' : 'halofit',#option for computing non-linear power spectum
        'P_k_max_1/Mpc':10.0,
        'output': 'mPk' #which quantities we want CLASS to compute
    }
    #initialize Class instance and set parameters
    cosmo = Class()
    cosmo.set(params)
    cosmo.compute()
    h = cosmo.h()
    sigma_8 = cosmo.sigma(8/cosmo.h(),0)
    return h, sigma_8

#as a cross-check, try to derive h, sigma_8 for abacus_cosm132
p = np.array([0.02237, 1.2000e-01, 2.1791e-09, 9.0490e-01])
print("om_b, om_c, A_s, n_s = ", p)
h,sigma_8 = derived_parameters(p)
print("h = ",h) #should be 0.6736
print("sigma_8 = ",sigma_8) #should be 0.808189 


om_b, om_c, A_s, n_s =  [2.2370e-02 1.2000e-01 2.1791e-09 9.0490e-01]
h =  0.6735945
sigma_8 =  0.80790416541136


<span style="color:blue"> *3e-4 fractional difference in $\sigma_8$ to $\text{abacus_cosm132}$, negligible compared to expectd sensitivity.*</span>

### Abacus Summit Parameter range

In [3]:
# johannes' code for Summit cosmology range
from scipy.linalg.lapack import dpotrf, dpotri

def invert_symmetric_positive_semidefinite_matrix(m):
    r"""Invert a symmetric positive sem-definite matrix. This function is
    faster than numpy.linalg.inv but does not work for arbitrary matrices.
    Attributes
    ----------
    m : numpy.ndarray
        Matrix to be inverted.
    Returns
    -------
    m_inv : numpy.ndarray
        Inverse of the matrix.
    """

    m_inv = dpotri(dpotrf(m, False, False)[0])[0]
    m_inv = np.triu(m_inv) + np.triu(m_inv, k=1).T
    return m_inv


def minimum_volume_enclosing_ellipsoid(points, tol=0, max_iterations=1000):
    r"""Find an approximation to the minimum bounding ellipsoid to:math:`m`
    points in :math:`n`-dimensional space using the Khachiyan algorithm.
    This function is based on
    http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.116.7691 but
    implemented independently of the corresponding MATLAP implementation or
    other Python ports of the MATLAP code.
    Attributes
    ----------
    points : numpy.ndarray with shape (m, n)
        A 2-D array where each row represents a point.
    Returns
    -------
    c : numpy.ndarray
        The position of the ellipsoid.
    A : numpy.ndarray
        The bounds of the ellipsoid in matrix form, i.e.
        :math:`(x - c)^T A (x - c) \leq 1`.
    """

    m, n = points.shape
    q = np.append(points, np.ones(shape=(m, 1)), axis=1)
    u = np.repeat(1.0 / m, m)
    q_outer = np.array([np.outer(q_i, q_i) for q_i in q])
    e = np.diag(np.ones(m))

    for i in range(max_iterations):
        if i % 1000 == 0:
            v = np.einsum('ji,j,jk', q, u, q)
        g = np.einsum('ijk,jk', q_outer,
                      invert_symmetric_positive_semidefinite_matrix(v))
        j = np.argmax(g)
        d_u = e[j] - u
        a = (g[j] - (n + 1)) / ((n + 1) * (g[j] - 1))
        shift = np.linalg.norm(a * d_u)
        v = v * (1 - a) + a * q_outer[j]
        u = u + a * d_u
        if shift <= tol:
            break

    c = np.einsum('i,ij', u, points)
    A_inv = (np.einsum('ji,j,jk', points, u, points) - np.outer(c, c)) * n
    A = np.linalg.inv(A_inv)

    scale = np.amax(np.einsum('...i,ij,...j', points - c, A, points - c))
    A /= scale

    return c, A
#read in omega_b, omega_cdm, A_s, n_s of summit cosmology grid
summit_cosmology_table = np.genfromtxt("summit_cosmologies.txt",delimiter=',')[:,[2,3,5,6]]
summit_h = np.genfromtxt("summit_cosmologies.txt",delimiter=',')[:,[4]]
# you can now draw the envelope with this
prior_c, prior_A = minimum_volume_enclosing_ellipsoid(summit_cosmology_table)

# check if a new parameter set p is within this envelope
def check_summit_range(p, prior_c, prior_A):
    return np.einsum('...i,ij,...j', p - prior_c, prior_A, p - prior_c) <= 1

### Generate parameter point within the range of all emulators
Draw $\omega_b,\omega_c, A_s, n_s$ within $\text{aemulus-}\nu$ and $\text{Summit}$ ranges, then calculate derived parameters and verify that the point is within h-range of $\text{aemulus-}\nu$ and within five-parameter range of $\text{BACCO}$ and $\text{Quijote}$.

In [4]:
def draw_4params_aemulus():
    r''' draw om_b, om_c, A_s, n_s within aemulus-𝜈 range
    return parameter vector with order [om_b, om_c, A_s, n_s]
    NOTE: need to check that theta_*-derived h is in aemulus range later!!
    '''
    #draw omega_b, omega_c within aemulus-𝜈 range
    om_b = np.random.uniform(low = 0.0198, high =  0.0248)
    om_c = np.random.uniform(low = 0.08, high =   0.16)
    #draw A_s, n_s within aemulus-𝜈 range 
    A_s = np.random.uniform(low = 1.1, high =  3.1)*1.e-9
    n_s = np.random.uniform(low = 0.93, high =   1.01)
    return np.array([om_b,om_c,A_s,n_s])

def p_in_aemulus_summit():
    r'''
    draw random p = [om_b, om_c, A_s, n_s] within aemulus + Abacus summit ranges
    '''
    p = draw_4params_aemulus()
    while (check_summit_range(p, prior_c, prior_A) == False):
        p = draw_4params_aemulus()
    return p

def check_BACCO_range(p, h,sigma_8):
    r'''check whether p and derived parameters are in BACCO range
    '''
    #Omega_m: BACCO narrower than Quijote
    O_m = (p[0]+p[1]+0.0006442)/h**2
    if (O_m < 0.23): 
        return False
    if (O_m > 0.4): 
        return False
    #Omega_b: BACCO narrower than Quijote
    O_b = (p[0])/h**2
    if (O_b <0.04):
        return False
    if (O_b> 0.06):
        return False
    #sigma_8: BACCO narrower than Quijote
    if (sigma_8 < 0.65):
        return False
    if (sigma_8 > 0.9):
        return False
    #H0: BACCO narrower than aemulus-𝜈 and Quijote
    if (h < 0.6):
        return False
    if (h > 0.8):
        return False
    return True

def p_in_range():
    p = p_in_aemulus_summit()
    h,sigma_8 = derived_parameters(p)
    while (check_BACCO_range(p,h,sigma_8) == False):
        p = p_in_aemulus_summit()
        h,sigma_8 = derived_parameters(p)
    return [p[0],p[1],p[2],p[3], h, sigma_8]
    
six_params = p_in_range()
print(six_params)

[0.021755216490503833, 0.11759420467151259, 1.6894494110020212e-09, 0.9724531807330381, 0.674202, 0.7237628139510479]
